In [37]:
from xml.etree import ElementTree
import pandas as pd
import os
from openai import OpenAI
from pydantic import BaseModel
from typing import List

In [38]:
import glob
files = glob.glob('data/*.xml')

patients = []
for file in files:
    with open(file, 'r') as f:
        patients.append(f.read())


# Example: Parse all records into a structured format
import xml.etree.ElementTree as ET

all_records = []

for patient_xml in patients:
    root = ET.fromstring(patient_xml)
    pacient = root.find('pacient')
    
    if pacient is not None:
        patient_id = pacient.get('id')
        
        for zaznam in pacient.findall('zaznam'):
            record = {
                'patient_id': patient_id,
                'date': zaznam.find('datum').text if zaznam.find('datum') is not None else None,
                'type': zaznam.find('typ').text if zaznam.find('typ') is not None else None,
                'text': zaznam.find('text').text if zaznam.find('text') is not None else None
            }
            all_records.append(record)

# Convert to DataFrame for easy analysis
df = pd.DataFrame(all_records)
print(f"Total records parsed: {len(df)}")
print(f"\nFirst few records:")
df.head()


Total records parsed: 335

First few records:


,patient_id,date,type,text
0,HACK03,2022-10-04,A-KONZ,První onkologická konzultace pro nově diagnost...
1,HACK03,2022-10-05,A-KF,Klinicko-farmaceutická konzultace. Zhodnocena ...
2,HACK03,2022-10-07,KOC,Hospitalizační epikríza – krátkodobý příjem na...
3,HACK03,2022-10-10,A-ONK3,Onkologická kontrola v průběhu paliativní syst...
4,HACK03,2022-10-17,A-ONK3,Onkologická kontrola v průběhu paliativní syst...


In [40]:
from tqdm import tqdm

class InformationList(BaseModel):
    information_list: List[str]

results = []

tqdm.pandas()
for idx, row in tqdm(df[df["patient_id"] == "HACK01"].sort_values(by="date", ascending=True).iterrows(), total=len(df[df["patient_id"] == "HACK01"])):

    input_text = row["text"]
    api_key = os.environ.get("OPENAI_API_KEY")

    client = OpenAI(api_key=api_key)
    response = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {
                "role": "developer",
                "content": """Jsi doktor/dokumentátor v nemocnici. Je ti zadán text. Diagnóza o pacientovi. Extrahuj všechny informace, na výstupu je seznam informací.
                Každá informace je atomická. Všechno jako oznamovací věty."""
            },
            {
                "role": "user",
                "content": input_text
            }
        ],
        text_format=InformationList
    )
    results.append(response.output_parsed)


  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [11:27<00:00, 42.98s/it]


In [45]:
full_list = []

for res in results:
    full_list.extend(res.information_list)
full_list

['Důvod odeslání je posouzení možností radioterapie.',
 'Odesílající lékař pochází z dermatologické ambulance regionální nemocnice.',
 'Pracoviště odesílajícího týmu je regionální nemocnice.',
 'Pacientka má dlouhodobý defekt kůže na temeni hlavy po dřívějším úrazu a plastické operaci.',
 'V posledním roce kožní léze zvětšuje.',
 'Kožní léze krvácí.',
 'Z rány vytéká zapáchající sekret.',
 'Pacientka má časté bolesti hlavy.',
 'Pacientka má únavu.',
 'Pacientka má obtížný spánek.',
 'Karnofskyho index je 70 procent.',
 'Pacientka je při vědomí.',
 'Pacientka je orientovaná.',
 'Pacientka je spolupracující.',
 'Pacientka má pokrývku hlavy.',
 'Na kalvě parietálně vlevo přesahující střední čáru je nepravidelné ulcerované ložisko.',
 'Ložisko měří přibližně 14 × 14 cm.',
 'Ložisko má šedožlutý povlak.',
 'Ložisko obsahuje místy granulační tkáň.',
 'V centru defektu je patrná destrukce kosti kalvy.',
 'Okolní kůže je ztenčená.',
 'Okolní kůže je zarudlá.',
 'Regionální uzliny nejsou hmatné

In [34]:
from pprint import pprint
pprint(input_text)

print(response.output_parsed)

('První onkologická konzultace pro nově diagnostikovaný karcinom levého prsu u '
 'pacientky po menopauze. Nález v horním zevním kvadrantu s kožní infiltrací a '
 'hmatnými uzlinami v axile. Subjektivně dominují dlouhodobé bolesti zad a '
 'kloubů při známé revmatoidní artritidě, chůze o francouzských holích. '
 'Provedeno základní fyzikální vyšetření a kontrola dokumentace z '
 'mamologického pracoviště. Závěr: suspektně lokálně pokročilý nádor prsu, '
 'nutné dovyšetření a kompletní staging.')
information_list=['Pacientka má nově diagnostikovaný karcinom levého prsu.', 'Nález byl v horním zevním kvadrantu levého prsu.', 'Byla přítomna kožní infiltrace.', 'Byly hmatné uzliny v axile.', 'Pacientka má revmatoidní artritidu.', 'Pacientka chodí o francouzských holích.', 'Bylo provedeno základní fyzikální vyšetření.', 'Provedena byla kontrola dokumentace z mamologického pracoviště.', 'Závěr zní, že suspektně lokálně pokročilý nádor prsu.', 'Bylo nutné dovyšetření.', 'Bylo nutné provést kom

In [42]:
from tqdm import tqdm

class InformationList(BaseModel):
    information_list: List[str]

results2 = []

tqdm.pandas()
for idx, row in tqdm(df[df["patient_id"] == "HACK02"].sort_values(by="date", ascending=True).iterrows(), total=len(df[df["patient_id"] == "HACK02"])):

    input_text = row["text"]
    api_key = os.environ.get("OPENAI_API_KEY")

    client = OpenAI(api_key=api_key)
    response = client.responses.parse(
        model="gpt-5-nano",
        input=[
            {
                "role": "developer",
                "content": """Jsi doktor/dokumentátor v nemocnici. Je ti zadán text. Diagnóza o pacientovi. Extrahuj všechny informace, na výstupu je seznam informací.
                Každá informace je atomická. Všechno jako oznamovací věty."""
            },
            {
                "role": "user",
                "content": input_text
            }
        ],
        text_format=InformationList
    )
    results2.append(response.output_parsed)


100%|██████████| 33/33 [11:15<00:00, 20.48s/it]
